## Local Inference

In [55]:
%matplotlib inline
import cv2, numpy as np, matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
import time

%matplotlib inline

In [56]:
def get_model(num_keypoints, weights_path=None):
    
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
                                                                   pretrained_backbone=True,
                                                                   num_keypoints=num_keypoints,
                                                                   num_classes = 2, # Background is the first class, object is the second class
                                                                   rpn_anchor_generator=anchor_generator)

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

In [57]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model(num_keypoints = 4, weights_path='./assets/keypoint_model/weights/keypointsrcnn_weights 001.pth')
model.to(device)

c:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and may be removed in the future, please use 'weights_backbone' instead.
  warnings.warn(
c:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or 

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, 

## Run model on Live Camera or Video Recording

In [135]:
# frame capture rate in seconds
capture_rate = 5

# keypoint, bbox and frame lists 
keypoints_main = []
bboxes_main = []

# video file
video_file = "./assets/pfd_video_dataset/20221014_124610.mp4"

### Test on video recording 
Choose from:
* Display video and draw over bounding box
* Extract frames for further processing (saves more time)

In [148]:
# with displaying video
frame_list = []
keypoint_list = []
bbox_list = []

# Open the video file
video = cv2.VideoCapture(video_file)

# Check if video is opened successfully
if not video.isOpened():
    print("Error opening video file")

# Get the width and height of the video
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set the desired width
desired_width = 500

# Calculate the aspect ratio
aspect_ratio = height / width

# Calculate the new height
desired_height = int(desired_width * aspect_ratio)

# Get the frames per second of the video
fps = video.get(cv2.CAP_PROP_FPS)

# Calculate the number of frames to skip
skip_frames = int(fps * capture_rate)

#### Display video and draw keypoints and bounding box

In [149]:
# Initialize the counter for frames
frame_count = 0

# Read until video is completed
while(video.isOpened()):
    # Capture frame-by-frame
    ret, frame = video.read()
    
    if not ret:
        break
    
    if frame_count % skip_frames == 0:
        frame_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)            
        frame_tensor = F.to_tensor(frame_np)
        frame_tensor = frame_tensor.to(device)
        
        images = [frame_tensor]

        with torch.no_grad():
            model.to(device)
            model.eval()
            output = model(images)
        
        frame_tensor = (images[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()
        
        high_scores_idxs = np.where(scores > 0.8)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

        keypoints = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append([list(map(int, kp[:2])) for kp in kps])
            
        bboxes = []
        for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            bboxes.append(list(map(int, bbox.tolist())))

        if(len(keypoints) != 0):
            keypoints_main = keypoints
        
        if(len(bboxes) != 0):
            bboxes_main = bboxes

        # add frame to to frame_list
        frame_list.append(cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB))
        keypoint_list.append(keypoints_main)
        bbox_list.append(bboxes_main)

    if(len(keypoints_main) != 0):
        points = np.array(keypoints_main, np.int32)
        points = points.reshape((-1, 1, 2))

        # Draw a keypoints on the frame
        # cv2.polylines(frame, [points], isClosed=True, color=(0, 0, 0), thickness=8)
        for point in points:
            center = (point[0][0], point[0][1])
            radius = 15

            # Draw the keypoints using the circle() function
            cv2.circle(frame, center, radius, (0, 0, 255), -1)

    if(len(bboxes_main) != 0):
        # draw a bbox on the frame
        cv2.rectangle(frame, (bboxes_main[0][0], bboxes_main[0][1]),  (bboxes_main[0][2], bboxes_main[0][3]), color=(0, 255, 0), thickness=8)

    # Display the resulting frame
    frame = cv2.resize(frame, (desired_width, desired_height), interpolation=cv2.INTER_AREA)
    cv2.imshow('Frame', frame)
    
    # Press Q on keyboard to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_count = frame_count + 1
# Release the video
video.release()

# Close all windows
cv2.destroyAllWindows()

#### Extract frames and keypoints for further processing

In [137]:
# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Loop through the frames of the video
for i in range(total_frames):
    # Capture frame-by-frame
    ret, frame = video.read()

    if i % skip_frames == 0:
        frame_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)            
        frame_tensor = F.to_tensor(frame_np)
        frame_tensor = frame_tensor.to(device)

        images = [frame_tensor]
        
        with torch.no_grad():
            model.to(device)
            model.eval()
            output = model(images)

        frame_tensor = (images[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()

        high_scores_idxs = np.where(scores > 0.8)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

        keypoints = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append([list(map(int, kp[:2])) for kp in kps])
            
        bboxes = []
        for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            bboxes.append(list(map(int, bbox.tolist())))

        # add frame to to frame_list
        frame_list.append(frame_tensor)
        keypoint_list.append([keypoints[0]])
        bbox_list.append(bboxes)

# Release the video
video.release()

### Test on live camera

In [4]:
# Initialize the counter for frames
frame_count = 0

# Create a VideoCapture object
cap = cv2.VideoCapture(0) # 0 means the default camera

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    if int(time.time()) % capture_rate == 0:
        frame_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_tensor = F.to_tensor(frame_np)
        frame_tensor = frame_tensor.to(device)
        images = [frame_tensor]


        with torch.no_grad():
            model.to(device)
            model.eval()
            output = model(images)
        
        frame_tensor = (images[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()
        
        high_scores_idxs = np.where(scores > 0.7)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

        keypoints = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append([list(map(int, kp[:2])) for kp in kps])
            
        bboxes = []
        for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            bboxes.append(list(map(int, bbox.tolist())))
    
        if(len(keypoints) != 0):
            print(keypoints)
            keypoints_main = keypoints
        
        if(len(bboxes) != 0):
            print(bboxes)
            bboxes_main = bboxes

        # add frame to to frame_list
        frame_list.append(frame)
        keypoint_list.append(keypoints_main)
        bbox_list.append(bboxes_main)

    if(len(keypoints_main) != 0):
        points = np.array(keypoints_main, np.int32)
        points = points.reshape((-1, 1, 2))

        # Draw a keypoints on the frame
        # cv2.polylines(frame, [points], isClosed=True, color=(0, 0, 0), thickness=8)
        for point in points:
            center = (point[0][0], point[0][1])
            radius = 15

            # Draw the keypoints using the circle() function
            cv2.circle(frame, center, radius, (255, 0, 0), -1)

    if(len(bboxes_main) != 0):
        # draw a bbox on the frame
        cv2.rectangle(frame, (bboxes_main[0][0], bboxes_main[0][1]),  (bboxes_main[0][2], bboxes_main[0][3]), color=(0, 255, 0), thickness=8)
        
    # Display the frame with the bounding box
    cv2.imshow("Live Video with Bounding Box", frame)

    # `Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count = frame_count + 1

# Release the video capture object
cap.release()

# Close all windows
cv2.destroyAllWindows()

## Perspective Transformation and Timelapse

#### Frame Filter: Algorithm 1 (based on slope angles)

In [153]:
filtered_keypoints = []
filtered_frames = []
# filter threshold
threshold = 2

# remove coordinates if its x or y is lower than the average + threshold
for count, kp in enumerate(keypoint_list):
    coordinates = kp[0]

    # Sort the coordinates based on x and y
    coordinates = sorted(coordinates)

    # Get the left and right side coordinates
    left_side_coordinates = coordinates[:2]
    right_side_coordinates = coordinates[2:]

    # Get the top and bottom side coordinates
    if left_side_coordinates[0][1] < left_side_coordinates[1][1]:
        top_side_coordinates = left_side_coordinates
        bottom_side_coordinates = right_side_coordinates
    else:
        top_side_coordinates = right_side_coordinates
        bottom_side_coordinates = left_side_coordinates

    # Extract the slope angles
    left_side_slope = 0 if top_side_coordinates[0][0] - top_side_coordinates[1][0] == 0 else (top_side_coordinates[0][1] - top_side_coordinates[1][1]) / (top_side_coordinates[0][0] - top_side_coordinates[1][0])
    right_side_slope = 0 if bottom_side_coordinates[0][0] - bottom_side_coordinates[1][0] == 0 else (bottom_side_coordinates[0][1] - bottom_side_coordinates[1][1]) / (bottom_side_coordinates[0][0] - bottom_side_coordinates[1][0])
    top_side_slope = 0 if top_side_coordinates[1][0] - top_side_coordinates[0][0] == 0 else (top_side_coordinates[1][1] - top_side_coordinates[0][1]) / (top_side_coordinates[1][0] - top_side_coordinates[0][0])
    bottom_side_slope = 0 if bottom_side_coordinates[1][0] - bottom_side_coordinates[0][0] == 0 else (bottom_side_coordinates[1][1] - bottom_side_coordinates[0][1]) / (bottom_side_coordinates[1][0] - bottom_side_coordinates[0][0])

    diff_top_bottom = abs(top_side_slope - bottom_side_slope)
    diff_left_right = abs(left_side_slope - right_side_slope)

    if diff_top_bottom < threshold and diff_left_right < threshold:
        filtered_keypoints.append(coordinates)
        filtered_frames.append(frame_list[count])
    
removed = len(keypoint_list) - len(filtered_frames)
print('Length of frames removed (threshold = {}): {}/{}'.format(threshold, removed, len(keypoint_list)))

Length of frames removed (threshold = 2): 30/33


#### Perspective Transformation and Timelapse Creation

In [155]:
# Calculate the frame rate
num_frames = len(filtered_frames)
timelapse_length = num_frames / 2
frame_rate = int(num_frames / timelapse_length)

# Define the codec and create a video writer
fourcc = cv2.VideoWriter_fourcc(*"MP4V")
out = cv2.VideoWriter("timelapse.mp4", fourcc, frame_rate, (desired_width, desired_height))

# Define the transformation matrix
keypoint = filtered_keypoints[0]

for count, frame in enumerate(frame_list):
    pt1 = np.float32([keypoint[1],keypoint[3],keypoint[0],keypoint[2]])
    pt2 = np.float32([[0,0],[desired_width,0],[0,desired_height],[desired_width,desired_height]])
    M = cv2.getPerspectiveTransform(pt1,pt2)

    # Perform the transformation
    warped_image = cv2.warpPerspective(frame, M, (desired_width, desired_height))
    warped_image = cv2.cvtColor(warped_image,cv2.COLOR_BGR2RGB)

    out.write(warped_image)

# Release the video writer
out.release()